# Transformers 를 이용해 Backbone 사용

## Transformers 설치
- `pip install transformers`

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [1]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [5]:
# model_id = "bert-base-uncased"
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))
model = AutoModel.from_pretrained(model_id)
print(type(model))
config = AutoConfig.from_pretrained(model_id)
print(type(config))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\Playdata\miniconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

<class 'transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast'>


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

<class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>


In [9]:
raw_txt = "I am a boy."
token = tokenizer(
    raw_txt,
    return_tensors="pt"  
    # 토큰화 결과: default-list, "pt"-torch.tensor, 'tf'-tensorflow.tensor, 'np'-np.ndarray
)
token
# input_ids: 토큰 id
# attention_mask: input_ids의 각 토큰들이 실제 토큰인지 [PAD] 토큰인지 여부. 1: 유효(실제) 문자열 토큰, 0: [PAD]

{'input_ids': tensor([[  40,  716,  257, 2933,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [10]:
type(token['input_ids'])

torch.Tensor

In [12]:
# model
context_vector = model(**token)

In [ ]:
context_vector.last_hidden_state.shape
# [1: 문서(문장) 개수, 5: 토큰수, 768: embedding vector 차원]


torch.Size([1, 5, 768])

In [21]:
# context_vector.past_key_values[0]

In [26]:
###### 토크나이저
t = tokenizer.encode("I am a boy.")  # token id들만 리턴.
t

[40, 716, 257, 2933, 13]

In [27]:
tokenizer.decode(t)

'I am a boy.'

In [33]:
# 개별 토큰 확인
tokenizer.convert_tokens_to_ids("am")
tokenizer.convert_tokens_to_ids(["i", "am", "a", "boy", "나는"])

[72, 321, 64, 7081, 50256]

In [34]:
tokenizer.convert_ids_to_tokens(321)
tokenizer.convert_ids_to_tokens([72, 321, 64, 7081, 50256])

['i', 'am', 'a', 'boy', '<|endoftext|>']

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저 모델 load

In [35]:
from transformers import AutoModel, AutoTokenizer

model_id = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

In [36]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(300, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### 입력값 토큰화

In [50]:
sentences = [
    "안녕",
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

In [51]:
token = tokenizer(sentences[0])
token

{'input_ids': [2, 19017, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [ ]:
token.keys()
# input_ids: 토큰 id들
# attention_mask: input_ids의 개별 토큰들이 실제 값인지 [PAD] 인지 구분. (1: 실제토큰, 0: [PAD])
# token_type_ids: 입력이 두개 문서로 구성된 경우(question-answering: Question, Context) 각 토큰이 몇번째 문서인지 구분하기위한 값.
### input_ids:      [q, q, q, q, q, q, c, c, c, c, c] 
### token_type_ids: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [41]:
token['token_type_ids']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [52]:
token_list = tokenizer(
    sentences,    # 토큰화할 문서들.
    max_length=10, # padding/truncation의 기준 길이 지정.
    padding=True, # 패딩 추가. default: batch에서 가장 긴 sequence에 맞춰서 패딩. 
                  # max_length가 설정된 경우는 max_length에 맞춘다.
    truncation=True, # max_length 보다 개수가 많은 sequence일 경우 max_length에 맞춰 뒤에 토큰들을 잘라낸다.
    return_tensors="pt"
)

In [54]:
token_list["input_ids"]

tensor([[    2, 19017,     3,     0,     0,     0,     0,     0,     0,     0],
        [    2,    41,  4223,  4403,  4403, 18940,    39,  4243,  4773,     3],
        [    2, 26182,  4113, 20684,  4130,  2451, 22088, 20002, 15999,     3]])

In [56]:
token_list['attention_mask']

tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [58]:
token_list['token_type_ids']

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [ ]:
output = model(**token_list)

In [62]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [65]:
output["pooler_output"].shape

torch.Size([3, 768])